In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from astropy import units as u
from astropy import time

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.plotting.static import StaticOrbitPlotter

import emcee
import corner
import timeit

In [2]:
R_earth = 6378.1 * u.km

# Data from ISS at 2019-apr-10
a    = R_earth + (409) * u.km
ecc  = 0.0001586 * u.one
inc  = 51.6418 * u.deg
raan = 332.5814 * u.deg
argp = 179.7333 * u.deg
nu   =-179.6181 * u.deg

In [3]:
EPOCH = time.Time('2019-04-10 12:00:00.0' , format='iso')

In [4]:
iss = Orbit.from_classical(Earth, a=a, ecc=ecc, inc=inc, raan=raan, argp=argp, nu=nu, epoch=EPOCH)

In [7]:
iss.epoch.unix

1554897600.0

In [8]:
iss.classical()

(<Quantity 6787.1 km>,
 <Quantity 0.0001586>,
 <Quantity 51.6418 deg>,
 <Quantity 332.5814 deg>,
 <Quantity 179.7333 deg>,
 <Quantity -179.6181 deg>)

| symbol | description |
|:--------:|------|
|   a | semi-major axis| 
| ecc | eccentricity|
| inc | inclination|
|raan | right ascension of the ascending node|
|argp | argument of the pericenter|
|  nu | true anomaly|

In [23]:
deltaT = 10 * u.second

positions = []

for i in range(100):
    aux = iss.propagate(deltaT * i)
    positions.append(aux)
    
    print("r", aux.r, "v", aux.v)

r [ 6029.52111627 -3118.34884883    10.70236677] km v [2.17594474 4.22796288 6.00833534] km / s
r [ 6050.89592872 -3075.87143146    70.78376217] km v [2.09897235 4.26743038 6.00781614] km / s
r [ 6071.49967168 -3033.00205386   130.85613754] km v [2.02173248 4.30635409 6.00653135] km / s
r [ 6091.32971951 -2989.74617884   190.91183783] km v [1.94423497 4.34472904 6.00448115] km / s
r [ 6110.38354511 -2946.10931846   250.9432101 ] km v [1.86648968 4.38255035 6.0016658 ] km / s
r [ 6128.65872024 -2902.0970333    310.94260449] km v [1.78850653 4.4198132  5.99808565] km / s
r [ 6146.15291583 -2857.71493176   370.90237524] km v [1.71029543 4.45651284 5.99374115] km / s
r [ 6162.86390231 -2812.96866936   430.81488161] km v [1.63186637 4.4926446  5.98863287] km / s
r [ 6178.78954985 -2767.86394798   490.67248887] km v [1.55322931 4.52820388 5.98276145] km / s
r [ 6193.92782863 -2722.40651519   550.46756929] km v [1.47439429 4.56318614 5.97612764] km / s
r [ 6208.27680914 -2676.60216348   610.1